# 1. 필요한 라이브러리 다운로드

In [ ]:
import pandas as pd
import re
from kiwipiepy import Kiwi
from gensim import corpora
from gensim.models import LdaModel
import pyLDAvis.gensim_models as gensimvis
import pyLDAvis
from tqdm import tqdm

# 2. 데이터셋 준비

In [ ]:
red = ['LeeSeungMan', 'YoonBoSun', 'ParkJeongHee', 'ChoiGyuHa', 'JeonDuHwuan',
             'RohTaeWoo', 'KimYoungSam', 'LeeMyungBak',
             'ParkGeunHye', 'YoonSeokYeol']

blue = ['KimDaeJung', 'RohMooHyun', 'MoonJaeIn']

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
# 텍스트 정제 함수 정의
def clean_text(text):
    text = text.replace(".", "").strip()
    text = text.replace("·", " ").strip()
    text = text.replace("\n", "").strip()
    pattern = '[^ ㄱ-ㅣ가-힣|0-9]+'
    text = re.sub(pattern=pattern, repl='', string=text)
    return text

# 명사 추출 함수
def get_nouns(tokenizer, sentence):
    tokens = tokenizer.analyze(sentence)[0][0]
    nouns = [word.form for word in tokens if word.tag in ['NNG', 'NNP'] and len(word.form) > 1]
    return nouns

# 3. LDA

In [ ]:
# 보수(red)와 진보(blue) 데이터셋 구분
red = ['LeeSeungMan', 'YoonBoSun', 'ParkJeongHee', 'ChoiGyuHa', 'JeonDuHwuan',
       'RohTaeWoo', 'KimYoungSam', 'LeeMyungBak', 'ParkGeunHye', 'YoonSeokYeol']

blue = ['KimDaeJung', 'RohMooHyun', 'MoonJaeIn']

# 데이터를 통합할 리스트 초기화
red_data = []
blue_data = []

# 데이터 로드 및 전처리
for name in red + blue:
    file_path = f'./dataset/cleaned_data/cleaned_{name}.csv'
    df = pd.read_csv(file_path)

    # 스피커에 따라 다른 컬럼 설정
    if name == 'YoonSeokYeol':
        df.columns = ["index", "title", "subtitle", "date", "speaker", "speech"]
    else:
        df.columns = ["index", "title", "type", "date", "speaker", "speech"]

    # 결측치 제거
    df = df.dropna(how='any')

    # 보수(red)와 진보(blue) 데이터로 분리
    if name in red:
        red_data.append(df)
    elif name in blue:
        blue_data.append(df)

# 보수와 진보 데이터를 각각 하나의 데이터프레임으로 합치기
red_df = pd.concat(red_data, ignore_index=True)
blue_df = pd.concat(blue_data, ignore_index=True)

# Kiwi 토크나이저 초기화
tokenizer = Kiwi()

# 연설문 토큰화
red_df['cleaned_speech'] = red_df['speech'].apply(lambda x: clean_text(x))
red_df['tokens'] = red_df['cleaned_speech'].apply(lambda x: get_nouns(tokenizer, x))

blue_df['cleaned_speech'] = blue_df['speech'].apply(lambda x: clean_text(x))
blue_df['tokens'] = blue_df['cleaned_speech'].apply(lambda x: get_nouns(tokenizer, x))

# 보수와 진보 데이터 각각에 대해 LDA 모델 학습
def run_lda(df, group_name):
    # 토픽 모델링 준비
    dictionary = corpora.Dictionary(df['tokens'])
    corpus = [dictionary.doc2bow(text) for text in df['tokens']]

    # LDA 모델 학습
    lda_model = LdaModel(corpus, num_topics=5, id2word=dictionary, passes=15)

    # 토픽 시각화
    vis = gensimvis.prepare(lda_model, corpus, dictionary)

    # pyLDAvis 시각화 저장
    pyLDAvis.save_html(vis, f'./lda/result/group_result/{group_name}_kiwi_lda.html')

    # 결과 출력
    for idx, topic in lda_model.print_topics(-1):
        print(f'Topic: {idx} \nWords: {topic}\n')

# 보수와 진보 데이터에 대해 각각 LDA 분석 수행
run_lda(red_df, "red")
run_lda(blue_df, "blue")

In [ ]:
# 보수와 진보 데이터를 합쳐서 분석하고 싶을 때:
combined_df = pd.concat([red_df, blue_df], ignore_index=True)
run_lda(combined_df, "combined")